In [4]:
from utils import bsh
import time
from datetime import datetime as dt
import sys
import traceback
from collections import OrderedDict

In [5]:
def cmdTraceback(e):
    eType, eValue, eTraceback = sys.exc_info()
    tbList = traceback.extract_tb(eTraceback)
    where = tbList[-1]

    return "%r at %s:%d" % (eValue, where[0], where[1])

In [6]:
def createSimData():
    sim = bsh('simulation', 'toto', 5432)
    sim.startComm()

    simData = {}

    for cmdStr in sim.cmdList:
        res = []

        for state in sim.states:
            sim.sendOneCommand('init')
            sim.sendOneCommand(state)
            stat = sim.sendOneCommand('status')

            ret = sim.sendOneCommand(cmdStr)
            newstat = int(sim.sendOneCommand('status'))
            res.append((state, (ret, newstat)))

        simData[cmdStr] = dict(res)

    return simData

In [7]:
def testBshState(simData, mode, host, port, ts=2):
    logger.info('==== STARTING BSH FUNCTIONAL TEST ====')
    try:
        ope = bsh(mode, host, port)
        ope.startComm()
        logger.info('==== BSH BOARD in mode %s  host : %s  port: %d ====\r\n'%(mode, host, port))
        for cmdStr in ope.cmdList:
            logger.info('==== TESTING CMD %s ====' % cmdStr)
            for state in ope.states:
                ope.sendOneCommand('init')
                time.sleep(ts)
                ope.sendOneCommand(state)
                time.sleep(ts)

                status = int(ope.sendOneCommand('status'))
                if status != ope.statList[state]:
                    logger.error('warning status (%d) should be %d' % (status, ope.statList[state]))

                time.sleep(ts)
                statword = int(ope.sendOneCommand('statword'))
                if statword != ope.statwords[state]:
                    logger.error('warning statword (%d) should be %d' % (statword, ope.statwords[state]))

                logger.info('testing on state %s ( status=%d )' % (state, status))
                time.sleep(ts)
                ret = ope.sendOneCommand(cmdStr)

                time.sleep(ts)
                newstat = int(ope.sendOneCommand('status'))

                if simData[cmdStr][state] != (ret, newstat):
                    logger.error('FAILED !!! ret : %sok, stat:%d'%(ret, newstat))
                else:
                    logger.info('OK ret : %sok, stat:%d'%(ret, newstat))
                    
                time.sleep(ts)
                
            logger.info('==== CMD %s VALIDATED ====\r\n' % cmdStr)

    except Exception as e:
        logger.error(cmdTraceback(e))
        raise

In [8]:
def testBiaConfig(mode, host, port, ts=2):
    testConf = {0 : dict(biaStrobe='off', biaDuty=255, biaPeriod=10000),
            1 : dict(biaStrobe=None, biaDuty=200, biaPeriod=None),
            2 : dict(biaStrobe=None, biaDuty=100, biaPeriod=None),
            3 : dict(biaStrobe=None, biaDuty=50, biaPeriod=None),
            4 : dict(biaStrobe='on', biaDuty=None, biaPeriod=None),
            5 : dict(biaStrobe=None, biaDuty=200, biaPeriod=None),
            6 : dict(biaStrobe=None, biaDuty=255, biaPeriod=1000),
            7 : dict(biaStrobe=None, biaDuty=None, biaPeriod=100),
            8 : dict(biaStrobe='off', biaDuty=255, biaPeriod=10000),
           }
    try:
        logger.info('==== STARTING BSH CONFIG TEST ====')
        
        ope = bsh(mode, host, port)
        ope.startComm()
        logger.info('==== BSH BOARD in mode %s  host : %s  port: %d ====\r\n'%(mode, host, port))
        
        state = 'bia_on'
        ope.sendOneCommand('init')
        time.sleep(ts)
        ope.sendOneCommand(state)
        time.sleep(ts)

        status = int(ope.sendOneCommand('status'))
        if status != ope.statList[state]:
            logger.error('warning status (%d) should be %d' % (status, ope.statList[state]))

        time.sleep(ts)
        statword = int(ope.sendOneCommand('statword'))
        if statword != ope.statwords[state]:
            logger.error('warning statword (%d) should be %d' % (statword, ope.statwords[state]))
        
        for kwargs in OrderedDict(sorted(testConf.items())).values():
            logger.info('==== TESTING CONF %s ===='%kwargs)
            ope.setBiaConfig(**kwargs)
            
            time.sleep(15)
            logger.info('==== TEST CONF OK ==== \r\n'%kwargs)
            
    except Exception as e:
        logger.error(cmdTraceback(e))
        raise

In [9]:
simData = createSimData()

In [11]:
import logging

logFolder ='/software/ait/logs/bsh'
#logFolder ='/home/arnaud/AIT-PFS/bshlogs'
logFile = '%s/%s-bshBoard-funcTest.log' % (logFolder, dt.utcnow().replace(microsecond=0).isoformat())

logger = logging.getLogger('bshBoard')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler(logFile)
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

In [12]:
host = 'arduitest.pfs'
port = 23
mode = 'simulation'

In [13]:
testBshState(simData, mode=mode, host=host, port=port)

In [14]:
testBiaConfig(mode=mode, host=host, port=port)